<a href="https://colab.research.google.com/github/superspray/DeepRecommender/blob/master/ML_FINAL_Hyperparameter_tuning1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/superspray/study.git

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from tqdm import tqdm

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import LinearSVC,SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif

train_data=pd.read_csv('/content/study/train2.csv')#, encoding='latin_1')#, engine='python', error_bad_lines=False)
#train_data = pd.read_csv('/content/Corona_NLP_train.csv', encoding='latin_1')

test_data = pd.read_csv('/content/study/Corona_NLP_test.csv')
train_data.columns=test_data.columns
print('train data shape is :', train_data.shape)
print('test data shape is :', test_data.shape)
train_data.tail()

#train_data.drop_duplicates(inplace= True)
train_data=train_data.drop_duplicates()
test_data.drop_duplicates(inplace=True)
train_data=train_data[train_data.Sentiment.notnull()]
#train_data = train_data[train_data['Sentiment'].notna()]
#train_data = train_data[train_data['Sentiment'].notna()]

print('train data shape is :', train_data.shape)
print('test data shape is :', test_data.shape)
train_data.tail()
train_data.head()


Cloning into 'study'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 90 (delta 33), reused 13 (delta 3), pack-reused 0
Unpacking objects: 100% (90/90), done.
train data shape is : (82311, 6)
test data shape is : (3798, 6)
train data shape is : (41158, 6)
test data shape is : (3798, 6)


,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,22249.0,67201.0,NaN,23-03-2020,"Donald J. Trump, I'm worried about the economi...",Positive
1,22250.0,67202.0,"Cranberry Township""",23-03-2020,Found my secret stash of #ToiletPaper for anyo...,Extremely Positive
2,39928.0,84880.0,California_USA,09-04-2020,joined the growing list of distilleries produ...,Extremely Positive
3,39929.0,84881.0,Worthing_England,09-04-2020,Spanish supermarket chains amp food distributo...,Negative
4,3799.0,48751.0,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral


In [2]:
import nltk
nltk.download('stopwords')
import re
from nltk.corpus import stopwords
stopWords = stopwords.words('english')


def clean(text):

    #     remove urls
    text = re.sub(r'http\S+', " ", str(text))
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"you'll", "you will", text)
    text = re.sub(r"i'll", "i will", text)
    text = re.sub(r"she'll", "she will", text)
    text = re.sub(r"he'll", "he will", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"there's", "there is", text)
    text = re.sub(r"here's", "here is", text)
    text = re.sub(r"who's", "who is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"shouldn't", "should not", text)
    text = re.sub(r"n't", " not", text)

    #     remove mentions
    text = re.sub(r'@\w+',' ',text)

    #     remove hastags
    text = re.sub(r'#\w+', ' ', text)

    #     remove digits
    text = re.sub(r'\d+', ' ', text)

    #     remove html tags
    text = re.sub('r<.*?>',' ', text)
    text = re.sub(r'[^(A-Za-z)]',r' ',text)
    text = re.sub('[\d]',r'',text)
    text = re.sub('[()]',r'',text)
    text = re.sub(r'(<.*?>)',r'',text)
    text = re.sub(r'  ',' ',text)
    #     remove stop words 
    text = text.split()
    text = " ".join([word.lower() for word in text if not word in stopWords])
    
    text = re.sub(r'  ',' ',text)
    text= re.sub(r"\s+"," ",text).strip()

    return text
train_df=train_data.copy().dropna(subset=['OriginalTweet']) 
test_df=test_data.copy()
train_df['OriginalTweet'] = train_df['OriginalTweet'].apply(lambda x: clean(x))
test_df['OriginalTweet'] = test_df['OriginalTweet'].apply(lambda x: clean(x))
train_df= train_df[train_df['OriginalTweet'].notna()]
train_df= train_df[train_df['OriginalTweet']!='']
train_df= train_df[train_df['OriginalTweet']!=' ']
train_df= train_df[train_df['OriginalTweet']!='  ']
test_df= test_df[test_df['OriginalTweet']!='']
test_df= test_df[test_df['OriginalTweet']!='']

print('train data shape is :', train_df.shape)
print('test df shape is :', test_df.shape)

train_df.head(10)


df_train = train_df.iloc[:,4:].reset_index(drop=True)
df_test = test_df.iloc[:,4:].reset_index(drop=True)

l = {"Neutral":0, "Positive":1,"Extremely Positive":1,"Negative":-1, "Extremely Negative":-1 }

df_train=df_train.replace({"Sentiment": l})
df_test=df_test.replace({"Sentiment": l})
df_train.head()
y_train = df_train['Sentiment'].copy()
y_test = df_test['Sentiment'].copy()
X_text_train = df_train['OriginalTweet'].copy()
X_text_test = df_test['OriginalTweet'].copy()
X_test=X_text_test
X_train=X_text_train

X_train.shape, y_train.shape,X_test.shape, y_test.shape

from sklearn.model_selection import train_test_split


from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
import datetime
### K-fold cross-validation using pipeline ###
from sklearn.model_selection import cross_val_score
import time
from sklearn.base import TransformerMixin 
from sklearn.feature_selection import SelectKBest, chi2
!pip install xgboost
#%%time
from xgboost import XGBClassifier

ngram=(1,1)
! pip install catboost
from catboost import CatBoostClassifier,Pool
# 시간 표시 함수

def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

class DenseTransformer(TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.toarray()
y_train.shape
from sklearn.model_selection import StratifiedKFold


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
train data shape is : (41119, 6)
test df shape is : (3796, 6)
     |████████████████████████████████| 66.3MB 63kB/s 


(41119,)

In [ ]:
X_train.shape

(41119,)

In [ ]:
### Grid search에 의한 초모수 결정 (SVM) ###
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

preproc = Pipeline([('tfidf',TfidfVectorizer(ngram_range = ngram,min_df = 5, max_df=0.6,lowercase = True)),('to_dense', DenseTransformer()),
                      ('selec',SelectKBest(k=1000,score_func=chi2))])
X_train1=preproc.fit_transform(X_train,y_train)
X_test1=preproc.transform(X_test)

param_range = [ 0.01, 0.1, 1.0, 10.0, 100.0]
param_grid = [{'C': param_range, 'loss' :['hinge', 'squared_hinge']}]
text_clf=LinearSVC(random_state=0, max_iter=5000)
gs = GridSearchCV(estimator=text_clf, param_grid=param_grid,
                scoring='accuracy', cv=5)
gs = gs.fit(X_train1, y_train)
print(gs.best_score_)
print(gs.best_params_)

clf = gs.best_estimator_
clf.fit(X_train1, y_train)
print(clf.score(X_train1,y_train))
print('Model Training Accuracy w/o CV: %.3f' % clf.score(X_train1, y_train))
print('Model Test Accuracy w/o CV: %.3f' % clf.score(X_test1, y_test))



/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

0.8654881220747918
{'C': 100.0, 'loss': 'hinge'}
0.8766020574430312
Model Training Accuracy w/o CV: 0.877
Model Test Accuracy w/o CV: 0.848


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


{'mean_fit_time': array([ 0.46330409,  0.50841064,  0.59043427,  0.51608262,  2.23697433,
         0.75002451,  5.74730797,  2.97400594,  9.37727318, 25.65161605]),
 'mean_score_time': array([0.0106554 , 0.01061983, 0.01065874, 0.01016645, 0.01036177,
        0.01064129, 0.01075687, 0.01054611, 0.01083136, 0.01051126]),
 'mean_test_score': array([0.65072097, 0.66677192, 0.72774158, 0.81286033, 0.84756456,
        0.85476315, 0.86432079, 0.85663578, 0.86548812, 0.8569276 ]),
 'param_C': masked_array(data=[0.01, 0.01, 0.1, 0.1, 1.0, 1.0, 10.0, 10.0, 100.0,
                    100.0],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_loss': masked_array(data=['hinge', 'squared_hinge', 'hinge', 'squared_hinge',
                    'hinge', 'squared_hinge', 'hinge', 'squared_hinge',
                    'hinge', 'squared_hinge'],
              mask=[False, False, False, Fa

In [ ]:
### Grid search에 의한 초모수 결정 (SVM) ###
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold

preproc = Pipeline([('tfidf',TfidfVectorizer(ngram_range = ngram,min_df = 5, max_df=0.6,lowercase = True)),('to_dense', DenseTransformer()),
                      ('selec',SelectKBest(k=1000,score_func=chi2))])
X_train1=preproc.fit_transform(X_train,y_train)
X_test1=preproc.transform(X_test)

param_grid = {'reg_alpha':[0,0.01],
'reg_alpha':[0,0.01],
'n_estimators': [100,200,500,1000],
'max_depth': [50,100],
'num_leaves': [30,60,100],
'min_data_in_leaf':[10,20]}


text_clf=lgb.LGBMClassifier(random_state=0,silent=True, verbose_eval=50)

gs = GridSearchCV(estimator=text_clf, param_grid=param_grid,
                scoring='accuracy', cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=0))
gs = gs.fit(X_train1, y_train)
print(gs.best_score_)
print(gs.best_params_)

clf = gs.best_estimator_
clf.fit(X_train1, y_train)
#print(clf.score(X_train1,y_train))
print('Model Training Accuracy w/o CV: %.3f' % clf.score(X_train1, y_train))
print('Model Test Accuracy w/o CV: %.3f' % clf.score(X_test1, y_test))
pd.DataFrame(gs.cv_results_)



In [ ]:
### Grid search에 의한 초모수 결정 (SVM) ###
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

preproc = Pipeline([('tfidf',TfidfVectorizer(ngram_range = ngram,min_df = 5, max_df=0.6,lowercase = True)),('to_dense', DenseTransformer()),
                      ('selec',SelectKBest(k=1000,score_func=chi2))])
X_train1=preproc.fit_transform(X_train,y_train)
X_test1=preproc.transform(X_test)

param_range = [ 0.01, 0.1, 1.0, 10.0, 100.0]
param_grid = [{'C': param_range, 'loss' :[ 'hinge','squared_hinge']}]
text_clf=LinearSVC(random_state=0, max_iter=5000)
gs = GridSearchCV(estimator=text_clf, param_grid=param_grid,
                scoring='accuracy', cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=0))
gs = gs.fit(X_train1, y_train)
print(gs.best_score_)
print(gs.best_params_)

clf = gs.best_estimator_
clf.fit(X_train1, y_train)
#print(clf.score(X_train1,y_train))
print('Model Training Accuracy w/o CV: %.3f' % clf.score(X_train1, y_train))
print('Model Test Accuracy w/o CV: %.3f' % clf.score(X_test1, y_test))

pd.DataFrame(gs.cv_results_)

In [ ]:
### Grid search에 의한 초모수 결정 (SVM) ###
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

preproc = Pipeline([('tfidf',TfidfVectorizer(ngram_range = ngram,min_df = 5, max_df=0.6,lowercase = True)),('to_dense', DenseTransformer()),
                      ('selec',SelectKBest(k=1000,score_func=chi2))])
X_train1=preproc.fit_transform(X_train,y_train)
X_test1=preproc.transform(X_test)

param_range = [ 0.01, 0.1, 1.0, 10.0, 100.0]
param_grid = [{'C': param_range, 'loss' :[ 'squared_hinge']}]
text_clf=LinearSVC(random_state=0, max_iter=5000)
gs = GridSearchCV(estimator=text_clf, param_grid=param_grid,
                scoring='accuracy', cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=0))
gs = gs.fit(X_train1, y_train)
print(gs.best_score_)
print(gs.best_params_)

clf = gs.best_estimator_
clf.fit(X_train1, y_train)
#print(clf.score(X_train1,y_train))
print('Model Training Accuracy w/o CV: %.3f' % clf.score(X_train1, y_train))
print('Model Test Accuracy w/o CV: %.3f' % clf.score(X_test1, y_test))

pd.DataFrame(gs.cv_results_)

In [ ]:
preproc = Pipeline([('tfidf',TfidfVectorizer(ngram_range = ngram,min_df = 5, max_df=0.6,lowercase = True)),('to_dense', DenseTransformer()),
                      ('selec',SelectKBest(k=1000,score_func=chi2))])
X_train1=preproc.fit_transform(X_train,y_train)
X_val1=preproc.transform(X_val)
train_dataset=Pool(data=X_train1, label=y_train)
eval_dataset=Pool(data=X_val1, label=y_val)

cat_cl=CatBoostClassifier(random_seed=0, max_depth=i, logging_level='Silent')
gs = GridSearchCV(estimator=cat_cl, param_grid=param_grid,
                scoring='accuracy', cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=0))
gs = gs.fit(X_train1, y_train)
print(gs.best_score_)
print(gs.best_params_)


clf = gs.best_estimator_
clf.fit(train_dataset,use_best_model=True)
#print(clf.score(X_train1,y_train))
print('Model Training Accuracy w/o CV: %.3f' % clf.score(train_dataset))
print('Model Test Accuracy w/o CV: %.3f' % clf.score(eval_dataset))
pd.DataFrame(gs.cv_results_)
